# Character Configuration

In [ ]:
import os
import zipfile
import shutil
from PIL import Image
import gradio as gr

# -----------------------------
# Configuration
# -----------------------------

DATA_IMG_DIR = "data/images"
DATA_CAP_DIR = "data/captions"
TEMP_DIR = "temp"

VALID_EXTS = (".jpg", ".jpeg", ".png", ".webp")

RESIZE_PRESETS = {
    "Face (512x512)": (512, 512),
    "Half Body (512x768)": (512, 768),
    "Full Body (768x1024)": (768, 1024)
}

os.makedirs(DATA_IMG_DIR, exist_ok=True)
os.makedirs(DATA_CAP_DIR, exist_ok=True)
os.makedirs(TEMP_DIR, exist_ok=True)

# -----------------------------
# Global State (Simple & Safe)
# -----------------------------

image_list = []
current_index = 0
character_name = "person"


# Function Directory

#### Zip file Extractor

In [ ]:
def extract_zip(zip_path):
    global image_list, current_index             # From out of the function

    if os.path.exists(TEMP_DIR):
        shutil.rmtree(TEMP_DIR)                  # Delete if already exist
    os.makedirs(TEMP_DIR, exist_ok=True)         # Create Temporary Folder

    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(TEMP_DIR)                   # Extract Zip file into Temporary Folder

    image_list = []
    for root, _, files in os.walk(TEMP_DIR):
        for f in files:
            if f.lower().endswith(VALID_EXTS):             # Extract only image
                image_list.append(os.path.join(root, f))   #

    image_list.sort()
    current_index = 0

    if len(image_list) == 0:
        raise ValueError("No valid images found in ZIP")

    return load_current_image()


#### Image Load

In [ ]:
def load_current_image():
    if not image_list:
        return None
    return Image.open(image_list[current_index]).convert("RGB")

def next_image():
    global current_index
    if current_index < len(image_list) - 1:
        current_index += 1
    return load_current_image()

def prev_image():
    global current_index
    if current_index > 0:
        current_index -= 1
    return load_current_image()

#### Saving Logics

In [ ]:

def save_processed(
    img,
    body_type,
    caption_extra,
    resize_preset,
    resize_method
):
    idx = str(current_index + 1).zfill(4)
    body_tag = body_type.lower().replace(" ", "")
    filename = f"{idx}_{character_name}_{body_tag}.jpg"

    # Resize
    if resize_preset != "No Resize":
        size = RESIZE_PRESETS[resize_preset]
        resample_map = {
            "Lanczos": Image.LANCZOS,
            "Bicubic": Image.BICUBIC,
            "Area": Image.BOX
        }
        img = img.resize(size, resample=resample_map[resize_method])

    img.save(os.path.join(DATA_IMG_DIR, filename), quality=95)

    caption = f"a photo of {character_name} person"
    if caption_extra.strip():
        caption += ", " + caption_extra.strip()

    with open(os.path.join(DATA_CAP_DIR, filename.replace(".jpg", ".txt")), "w") as f:
        f.write(caption)

    return f"Saved {filename}"

# Gradio Interface

In [ ]:

with gr.Blocks(title="Identity LoRA Dataset Builder") as app:
    gr.Markdown("## LoRA Dataset Builder")

    with gr.Row():
        zip_input = gr.File(label="Upload ZIP (images only)", file_types=[".zip"])
        name_input = gr.Textbox(label="Character Name")

    load_btn = gr.Button("Load Dataset")

    with gr.Row():
        image_display = gr.Image(label="Current Image", interactive=True)
        with gr.Column():
            body_type = gr.Radio(
                ["Face", "Half", "Full"],
                label="Body Type",
                value="Face"
            )
            caption_extra = gr.Textbox(
                label="Optional Caption Add-on",
                placeholder="standing, natural light"
            )
            resize_preset = gr.Dropdown(
                ["No Resize"] + list(RESIZE_PRESETS.keys()),
                label="Resize Preset",
                value="Face (512x512)"
            )
            resize_method = gr.Radio(
                ["Lanczos", "Bicubic", "Area"],
                label="Resize Method",
                value="Lanczos"
            )

    with gr.Row():
        prev_btn = gr.Button("Previous")
        next_btn = gr.Button("Next")
        save_btn = gr.Button("Save Image + Caption")

    status = gr.Textbox(label="Status", interactive=False)

    # -----------------------------
    # Bindings
    # -----------------------------

    def set_character_name(name):
        global character_name
        character_name = name.strip().lower()

    name_input.change(set_character_name, name_input, None)

    load_btn.click(extract_zip, zip_input, image_display)
    next_btn.click(next_image, None, image_display)
    prev_btn.click(prev_image, None, image_display)
    save_btn.click(
        save_processed,
        [image_display, body_type, caption_extra, resize_preset, resize_method],
        status
    )

app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://805116c2aaa2c9a6cd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Debugging

In [4]:
import os
import zipfile
import shutil
from PIL import Image, ImageOps
import gradio as gr
import numpy as np

# -----------------------------
# Configuration
# -----------------------------
DATA_IMG_DIR = "data/images"
DATA_CAP_DIR = "data/captions"
TEMP_DIR = "temp"
VALID_EXTS = (".jpg", ".jpeg", ".png", ".webp")

# Presets for LoRA training
RESIZE_PRESETS = {
    "Face (512x512)": (512, 512),
    "Half Body (512x768)": (512, 768),
    "Full Body (768x1024)": (768, 1024),
    "Square (1024x1024)": (1024, 1024)
}

os.makedirs(DATA_IMG_DIR, exist_ok=True)
os.makedirs(DATA_CAP_DIR, exist_ok=True)
os.makedirs(TEMP_DIR, exist_ok=True)

# -----------------------------
# Global State
# -----------------------------
image_list = []
current_index = 0
character_name = "person"

# -----------------------------
# Utility Functions
# -----------------------------
def extract_zip(zip_file_obj):
    global image_list, current_index
    if zip_file_obj is None:
        return None, "Error: No ZIP file uploaded.", "N/A"

    print(f"[DEBUG] Extracting ZIP: {zip_file_obj.name}")

    try:
        # Clear and recreate temp directory
        if os.path.exists(TEMP_DIR):
            shutil.rmtree(TEMP_DIR)
        os.makedirs(TEMP_DIR, exist_ok=True)

        with zipfile.ZipFile(zip_file_obj.name, 'r') as z:
            z.extractall(TEMP_DIR)

        # Collect images
        image_list = []
        for root, _, files in os.walk(TEMP_DIR):
            for f in files:
                if f.lower().endswith(VALID_EXTS):
                    image_list.append(os.path.join(root, f))

        image_list.sort()
        current_index = 0

        if len(image_list) == 0:
            return None, "Error: No valid images found in ZIP.", "N/A"

        img = load_current_image()
        dims = f"{img.width} x {img.height}" if img else "N/A"
        return img, f"Loaded {len(image_list)} images.", dims

    except Exception as e:
        return None, f"Error during extraction: {str(e)}", "N/A"

def load_current_image():
    if not image_list or current_index >= len(image_list):
        return None
    try:
        img_path = image_list[current_index]
        # Auto-orient handles rotation EXIF data which is common in photos
        img = Image.open(img_path).convert("RGB")
        img = ImageOps.exif_transpose(img)
        return img
    except Exception as e:
        print(f"[ERROR] Failed to load image: {e}")
        return None

def next_image():
    global current_index
    if image_list and current_index < len(image_list) - 1:
        current_index += 1
    return load_current_image()

def prev_image():
    global current_index
    if image_list and current_index > 0:
        current_index -= 1
    return load_current_image()

def set_character_name(name):
    global character_name
    character_name = name.strip().lower() or "person"

# -----------------------------
# Display Logic (Dynamic Pixel Sizes)
# -----------------------------
def get_dims(img_input):
    """
    Parses the ImageEditor input to show dimensions.
    Returns: (Original Dims String, Current Crop Dims String)
    """
    if img_input is None:
        return "N/A", "N/A"

    # ImageEditor returns a dict: {'background': ..., 'layers': ..., 'composite': ...}
    # 'composite' is the image with edits (crops) applied.
    # 'background' is the original loaded image.

    bg = img_input.get("background")
    comp = img_input.get("composite")

    orig_str = f"{bg.width} x {bg.height}" if bg else "N/A"

    # If user hasn't cropped yet, composite might be same as background or None
    # We prioritize showing the composite size if it exists
    current = comp if comp is not None else bg
    curr_str = f"{current.width} x {current.height}" if current else "N/A"

    return orig_str, curr_str

# -----------------------------
# Save Logic
# -----------------------------
def save_processed(img_input, body_type, caption_extra, resize_preset, resize_method):
    if img_input is None:
        return "Error: No image loaded."
    if not image_list:
        return "Error: No dataset loaded."

    # 1. Extract the actual image from ImageEditor dictionary
    # 'composite' holds the cropped/edited version.
    # If no edit made, it falls back to 'background'.
    img = img_input.get("composite")
    if img is None:
        img = img_input.get("background")

    if img is None:
        return "Error: Image data missing."

    # Convert from RGBA (common in editor) to RGB
    final_img = img.convert("RGB")

    debug_log = [f"[DEBUG] Processing Image {current_index+1}"]
    debug_log.append(f"[DEBUG] Post-Crop Size: {final_img.size}")

    try:
        idx = str(current_index + 1).zfill(4)
        body_tag = body_type.lower().replace(" ", "")
        filename = f"{idx}_{character_name}_{body_tag}.jpg"

        # 2. Apply Resize Preset
        if resize_preset != "No Resize":
            if resize_preset in RESIZE_PRESETS:
                target_size = RESIZE_PRESETS[resize_preset]

                # Resample filter mapping
                resample_map = {
                    "Lanczos": Image.Resampling.LANCZOS,
                    "Bicubic": Image.Resampling.BICUBIC,
                }
                f_method = resample_map.get(resize_method, Image.Resampling.LANCZOS)

                final_img = final_img.resize(target_size, resample=f_method)
                debug_log.append(f"[DEBUG] Resized to {target_size} using {resize_method}")
            else:
                debug_log.append(f"[DEBUG] Unknown preset: {resize_preset}, skipping resize.")

        # 3. Save Image
        save_path = os.path.join(DATA_IMG_DIR, filename)
        final_img.save(save_path, quality=95)

        # 4. Save Caption
        caption = f"a photo of {character_name} person"
        if caption_extra and caption_extra.strip():
            caption += ", " + caption_extra.strip()

        cap_path = os.path.join(DATA_CAP_DIR, filename.replace(".jpg", ".txt"))
        with open(cap_path, "w", encoding="utf-8") as f:
            f.write(caption)

        return f"Saved: {filename}\nDims: {final_img.size}\n" + "\n".join(debug_log)

    except Exception as e:
        return f"Error saving: {str(e)}"

# -----------------------------
# Gradio Interface
# -----------------------------
with gr.Blocks(theme=gr.themes.Soft(), title="LoRA Dataset Builder") as app:
    gr.Markdown("## ✂️ Identity LoRA Dataset Builder with Cropper")

    # --- Top Control Bar ---
    with gr.Row():
        with gr.Column(scale=1):
            zip_input = gr.File(label="1. Upload ZIP", file_types=[".zip"])
            load_btn = gr.Button("Load Images", variant="primary")
        with gr.Column(scale=2):
            name_input = gr.Textbox(label="Character Trigger Word", value="anshu")
            status = gr.Textbox(label="System Status", lines=2)

    gr.HTML("<hr>")

    # --- Main Editor Area ---
    with gr.Row():
        # Left: Image Editor
        with gr.Column(scale=3):
            # interactive=True enables the cropping tool inside the image
            image_editor = gr.ImageEditor(
                label="Image Editor (Crop Here)",
                type="pil",
                crop_size=None, # Allows freeform cropping
                interactive=True,
                height=600
            )

            # Pixel Display Area
            with gr.Row():
                orig_dims = gr.Textbox(label="Original Size", value="N/A", interactive=False)
                curr_dims = gr.Textbox(label="Current Crop Size", value="N/A", interactive=False)

        # Right: Settings & Save
        with gr.Column(scale=1):
            gr.Markdown("### 2. Settings")
            body_type = gr.Radio(["Face", "Half Body", "Full Body"], label="Type", value="Face")

            # Using Presets as requested in original code, but applied AFTER crop
            resize_preset = gr.Dropdown(
                ["No Resize"] + list(RESIZE_PRESETS.keys()),
                label="Target Output Size",
                value="Face (512x512)"
            )
            resize_method = gr.Radio(["Lanczos", "Bicubic"], label="Filter", value="Lanczos")

            caption_extra = gr.Textbox(label="Caption Tags", placeholder="e.g. smiling, blue shirt")

            save_btn = gr.Button("💾 Save Image + Caption", variant="primary", size="lg")

            gr.Markdown("### Navigation")
            with gr.Row():
                prev_btn = gr.Button("⬅️ Prev")
                next_btn = gr.Button("Next ➡️")

    # -----------------------------
    # Event Wiring
    # -----------------------------

    # 1. Load ZIP
    load_btn.click(
        fn=extract_zip,
        inputs=zip_input,
        outputs=[image_editor, status, orig_dims]
    )

    # 2. Navigation
    # We update editor AND the dimension text when moving images
    prev_btn.click(prev_image, None, image_editor)
    next_btn.click(next_image, None, image_editor)

    # 3. Dynamic Dimension Updates
    # Whenever the image content changes (loaded or edited/cropped), update the text boxes
    image_editor.change(
        fn=get_dims,
        inputs=image_editor,
        outputs=[orig_dims, curr_dims]
    )

    # 4. Save
    save_btn.click(
        fn=save_processed,
        inputs=[image_editor, body_type, caption_extra, resize_preset, resize_method],
        outputs=status
    )

    # 5. Name Update
    name_input.change(set_character_name, name_input, None)

# -----------------------------
# Launch
# -----------------------------
app.launch(share=True) # share=True generates a public link

/tmp/ipython-input-4237160282.py:198: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="LoRA Dataset Builder") as app:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://831afed77d570acd31.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [2]:
import gradio as gr
from PIL import Image
import numpy as np

def get_original_size(input_data):
    """
    Triggered when an image is uploaded.
    Returns the pixel dimensions of the original image.
    """
    if input_data is None:
        return "Waiting for upload...", 100, 100

    # ImageEditor returns a dictionary. We look at the 'background' (original)
    # or 'composite' depending on state, but usually 'background' is the base.
    # Note: If passing plain Image component, input_data is just the image.
    # Here we use ImageEditor, so input_data is a dictionary.

    image = input_data["background"]
    if image is None:
        return "Waiting for upload...", 100, 100

    w, h = image.size
    return f"Original Size: {w} x {h} pixels", w, h

def process_image(input_data, new_width, new_height):
    """
    Triggered when the Resize button is clicked.
    Takes the cropped image from the editor, resizes it, and returns the result.
    """
    if input_data is None:
        return None, "No image to process"

    # In Gradio's ImageEditor, the 'composite' key holds the image
    # as it looks in the editor (after user crops/draws).
    image = input_data["composite"]

    if image is None:
        # Fallback if no edits were made, use background
        image = input_data["background"]

    if image is None:
        return None, "No image found."

    # Perform Resizing
    # converting to int because sliders might return floats
    resized_img = image.resize((int(new_width), int(new_height)))

    final_w, final_h = resized_img.size
    return resized_img, f"Final Size: {final_w} x {final_h} pixels"

# --- Building the Interface ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## ✂️ Interactive Image Cropper & Resizer")

    with gr.Row():
        # Left Column: Input and Controls
        with gr.Column():
            gr.Markdown("### 1. Upload & Crop")
            # ImageEditor allows the user to crop inside the UI
            input_img = gr.ImageEditor(
                type="pil",
                label="Editor",
                crop_size=None, # Allows freeform cropping
                interactive=True
            )

            # Display Original Dimensions
            original_info = gr.Textbox(label="Original Dimensions", value="Waiting for upload...")

            gr.Markdown("### 2. Resize Settings")
            with gr.Row():
                width_slider = gr.Number(label="Target Width", value=500)
                height_slider = gr.Number(label="Target Height", value=500)

            process_btn = gr.Button("Apply Crop & Resize", variant="primary")

        # Right Column: Output
        with gr.Column():
            gr.Markdown("### 3. Result")
            output_img = gr.Image(type="pil", label="Final Image")
            final_info = gr.Textbox(label="Final Dimensions")

    # --- Event Wiring ---

    # When image is uploaded to editor, calculate original size
    input_img.change(
        fn=get_original_size,
        inputs=input_img,
        outputs=[original_info, width_slider, height_slider]
    )

    # When button is clicked, process the image
    process_btn.click(
        fn=process_image,
        inputs=[input_img, width_slider, height_slider],
        outputs=[output_img, final_info]
    )

# Launch the app directly in the notebook
demo.launch()

/tmp/ipython-input-640698456.py:52: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4222b051ddcae26532.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
